# Исследование надежности заемщиков


**Содержание** 
<a id=0> </a>

[Введение](#1)  
[Предобработка данных](#2)  
[Ответы на вопросы](#3)  
[Вывод](#4)

## Введение
<a id=1> </a>

Мы проводим исследование для того, чтобы помочь банку выяснить, как влияет семейное положение и количество детей на способность клиента вовремя погасить кредит.

Для решения данной задачи мы будем использовать данные о клиентах банка, предоставленные нам заказчиком. Для начала мы обработаем возможные ошибки в датасете, апотом будем изучать различные зависимости погашения долга в срок от семейного статуса, дохода и количества детей клиентов.

In [1]:
#импортируем нужные библиотеки
import pandas as pd 
import math 

In [2]:
try:
    bank_clients = pd.read_csv('/datasets/data.csv')
except FileNotFoundError:
    bank_clients = pd.read_csv('/Users/09e6y/Desktop/Катькис/Практикум/Проект 2 credit value/dataset.csv')
    
bank_clients #прочтем датасет

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [3]:
bank_clients.info() # общая информация о датасете

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных
<a id=2> </a>


1. В столбце с количеством рабочих дней встречаются отрицательные числа
2. Есть пропуски в столбцах days_employed и total_income.


### Заполнение пропусков

Для подсчетов пропусков в колонке 'days_emoloyed' воспользуемся конструкцией try-except.

In [4]:
absence_count = 0 #создадим переменную-счетчик, в которую будем собирать пропуски

for absence in bank_clients['days_employed']:
    try:
        bank_clients[bank_clients['days_employed'].isna].index() #эта ветка возвращает индекс элемента с пустым значением
    except:
        absence_count+=math.isnan(absence) #эта ветка добавит к счетчику 1 если найдет пропуск в исследуемом столбце
absence_count #выведем результат на экран

2174

In [5]:
(bank_clients['total_income']).isna().sum() #подсчитаем пропуски в столбце total_income другим способом

2174

Количество пропусков в обоих столбцах совпадает, возможно, это пропуски в данных одних и тех же клиентов. 
Кстати, мы можем это проверить, возможно эта информация будет полезна в будущем.
Судя по всему, пропуск в данных связан с тем что эти клиенты - безработные, ведь данных нет в столбце с количеством рабочих дней и с общим доходом - а это не похоже на случайную ошибку.


In [6]:
d_e = bank_clients[bank_clients['days_employed'].isna()==True].index #соберем индексы пропущенных значений в 'days_employed'
t_i = bank_clients[bank_clients['total_income'].isna()==True].index #соберем индексы пропущенных значений в 'total_income
if d_e.all() == t_i.all(): #сравним полученные списки, и если они равны - распечатаем вывод
    print('Пропуски в одних и тех же строках')

Пропуски в одних и тех же строках


Вычислим долю пропусков от общего числа значений.

In [7]:
bank_clients.isna().mean()

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

Получается, доля пропусков в интересующих нас столбцах составляет одну десятую часть - что может довольно существенно искажать данные, поэтому нам следует избавиться от этих пропусков. Данные для столбца total_income считать не обязательно, потому что количество пропусков в нем совпадает с количеством пропусков в столбце days_employed, а значит и доля пропусков будет такая же. 

### Проверка данных на аномалии и исправления.

In [8]:
bank_clients['days_employed'] = abs(bank_clients['days_employed']) #избавимся от отрицательных значений 
#заполним пропуски медианными значениями в обоих столбцах
bank_clients['days_employed'] = bank_clients['days_employed'].fillna(bank_clients['days_employed'].median()) 
bank_clients['total_income'] = bank_clients['total_income'].fillna(bank_clients['total_income'].median()) 
bank_clients.isna().mean() # выполним проверку

children            0.0
days_employed       0.0
dob_years           0.0
education           0.0
education_id        0.0
family_status       0.0
family_status_id    0.0
gender              0.0
income_type         0.0
debt                0.0
total_income        0.0
purpose             0.0
dtype: float64

Стаж очевидно не может быть отрицательным значением, поэтому просто переводим все отрицательные значения в столбце days_employed в положительные.

Мы заполнили пропуски медианным значением, что при работе с количественными переменными является более оптимальным, т.к. при использовании среднего значения мы получим искаженные в бОльшую сторону данные.

### Изменение типов данных.

In [9]:
bank_clients['total_income'] = bank_clients['total_income'].astype('int') #переведем данные в столбце total_income в цельночисленные
bank_clients['total_income'] #проверим что все получилось

0        253875
1        112080
2        145885
3        267628
4        158616
          ...  
21520    224791
21521    155999
21522     89672
21523    244093
21524     82047
Name: total_income, Length: 21525, dtype: int64

### Удаление дубликатов

Проверим, встречаются ли в датасете стркоки-дубликаты.

In [10]:
bank_clients.duplicated().sum() #посчитаем количество полных дубликатов

54

In [11]:
bank_clients = bank_clients.drop_duplicates().reset_index(drop=True) #избавимся от дубликатов
bank_clients.info() #убедимся что они удалены

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21471 non-null  int64  
 1   days_employed     21471 non-null  float64
 2   dob_years         21471 non-null  int64  
 3   education         21471 non-null  object 
 4   education_id      21471 non-null  int64  
 5   family_status     21471 non-null  object 
 6   family_status_id  21471 non-null  int64  
 7   gender            21471 non-null  object 
 8   income_type       21471 non-null  object 
 9   debt              21471 non-null  int64  
 10  total_income      21471 non-null  int64  
 11  purpose           21471 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


Теперь отыщем неявные дубликаты. Например, одни и те же значения могут быть записаны с заглавной или маленькой буквы.

In [12]:
object_columns = ['education', 'family_status', 'gender', 'income_type', 'purpose'] #создадим список колонок, в которых лежит текстовый тип данных
for item in object_columns:
   bank_clients[item] = bank_clients[item].str.lower() #приведем текстовый тип данных к единому регистру
    


In [13]:
bank_clients.duplicated().sum() #вновь проверим количество дубликатов

17

In [14]:
bank_clients = bank_clients.drop_duplicates().reset_index(drop=True) #избавимся от дубликатов
bank_clients.info() #убедимся что они удалены

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  int64  
 11  purpose           21454 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


Проверим, не встречаются ли в датасете еще какие-то аномалии или дубликаты

In [15]:
for item in bank_clients: 
    display(bank_clients[item].unique()) #соберем все уникальные значения

array([ 1,  0,  3,  2, -1,  4, 20,  5])

array([8437.67302776, 4024.80375385, 5623.42261023, ..., 2113.3468877 ,
       3112.4817052 , 1984.50758853])

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

array([0, 1, 2, 3, 4])

...

array(['f', 'm', 'xna'], dtype=object)

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

array([0, 1])

array([253875, 112080, 145885, ...,  89672, 244093,  82047])

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

1. В колонке с количеством детей есть отрицательное значение и есть человек с 20 детьми (примем это тоже за аномалию)
2. В колонке с полом есть аномалии - значение 'xna'.
3. В колонке 'perpuse' есть дубликаты, например 'заняться высшим образованием' и 'высшее образование'. Так как в дальнейшем мы хотим заняться категоризацией целей кредита, возможно, стоит скорректировать этот список на данном этапе.

In [16]:
bank_clients['children'] = abs(bank_clients['children']) #избавимся от отрицательных значений
bank_clients.loc[bank_clients['children'] == 20, 'children'] = 2 #поменяем количество детей с 20 на 2
display(bank_clients['children'].unique()) #проверим результат

array([1, 0, 3, 2, 4, 5])

In [17]:
bank_clients.loc[bank_clients['gender'] == 'xna'] #выявим строки с аномальным значением гендера

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10684,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,xna,компаньон,0,203905,покупка недвижимости


Такая строка всего 1, из датасета невозможно определить какого пола данный клиент банка, поэтому просто удалим данную строку.

In [18]:
bank_clients.drop(labels=[10684],axis=0, inplace=True)


Займемся данными из колонки purpose.

In [19]:
display(bank_clients['purpose'].unique()) #выведем на экран уникальные значения столбца 'purpose'
display(len(bank_clients['purpose'].unique())) #посчитаем их

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

38

In [20]:
bank_clients['purpose'] = bank_clients['purpose'].replace('сыграть свадьбу', 'свадьба')
bank_clients['purpose'] = bank_clients['purpose'].replace('на проведение свадьбы', 'свадьба')
bank_clients['purpose'] = bank_clients['purpose'].replace('получение высшего образования', 'высшее образование')
bank_clients['purpose'] = bank_clients['purpose'].replace('заняться высшим образованием', 'высшее образование')

display(len(bank_clients['purpose'].unique()))


34

Теперь мы точно избавились от всех явных и неявных дубликатов и аномалий. Как видно из наших данных, причиной появления ошибок и дубликатов, как правило, является человеческий фактор. Возможно если бы человек выбирал цель кредита из предложенного списка и просто ставил галочку - мы получили бы более однородные данные. Но для дальнейшей работы мы можем использовать и такие. 

### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма

Для начала разберемся, какому уровню образования какой идентификатор соответствует. 

In [21]:
bank_education = bank_clients.pivot_table(index=['education'],values=['education_id']) #построим сводную таблицу
bank_education = bank_education.sort_values(by='education_id') #отсортируем данные
bank_education #посмотрим на результат

,education_id
education,
высшее,0
среднее,1
неоконченное высшее,2
начальное,3
ученая степень,4


Тоже самое проделаем с семейным положением.

In [22]:
bank_family_status = bank_clients.pivot_table(index=['family_status'],values=['family_status_id']) #построим сводную таблицу
bank_family_status = bank_family_status.sort_values(by='family_status_id') #отсортируем данные
bank_family_status #посмотрим на результат

,family_status_id
family_status,
женат / замужем,0
гражданский брак,1
вдовец / вдова,2
в разводе,3
не женат / не замужем,4


Теперь удалим из исходного датасета столбцы 'education' и 'family_status' и оставим только их идентификаторы.

In [23]:
bank_clients = bank_clients.drop(columns=['education', 'family_status']).reset_index(drop=True)#удалим столбцы, перезапустим нумерацию
bank_clients.head(10) #выведем на экран верхние 10 строк

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,f,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,f,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,m,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,m,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,f,пенсионер,0,158616,свадьба
5,0,926.185831,27,0,1,m,компаньон,0,255763,покупка жилья
6,0,2879.202052,43,0,0,f,компаньон,0,240525,операции с жильем
7,0,152.779569,50,1,0,m,сотрудник,0,135823,образование
8,2,6929.865299,35,0,1,f,сотрудник,0,95856,свадьба
9,0,2188.756445,41,1,0,m,сотрудник,0,144425,покупка жилья для семьи


### Категоризация дохода

In [24]:
def categ_income(income): #напишем функцию, которая будет распределять клиентов по категориям в зависимости от их дохода
    if 0 < income < 30000:
        return 'E'
    if 30001 < income < 50000:
        return'D'
    if 50001 < income < 200000:
        return'C'
    if 200001 < income < 1000000:
        return'B'
    return'A'
display(categ_income(75000)) #проверим работоспособность функции
categ_income (750000)

'C'

'B'

In [25]:
bank_clients['total_income_category'] = bank_clients['total_income'].apply(categ_income) #применим функцию к датафрейму и создадим новый столбец
bank_clients.head(10) #посмотрим 10 верхних строк датафрейма, чтобы убедиться что все получилось

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,f,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,f,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,m,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,m,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,1,1,f,пенсионер,0,158616,свадьба,C
5,0,926.185831,27,0,1,m,компаньон,0,255763,покупка жилья,B
6,0,2879.202052,43,0,0,f,компаньон,0,240525,операции с жильем,B
7,0,152.779569,50,1,0,m,сотрудник,0,135823,образование,C
8,2,6929.865299,35,0,1,f,сотрудник,0,95856,свадьба,C
9,0,2188.756445,41,1,0,m,сотрудник,0,144425,покупка жилья для семьи,C


### Категоризация целей кредита

Разделим все кредиты на 4 группы в зависимости от целей кредита.

In [26]:
def credit_purpose(purpose): #напишем функцию, которая разделит цели кредита на категории
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    if ('жиль' in purpose)|('недвиж' in purpose):
        return 'операции с недвижимостью'
    if  'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образован' in purpose:
        return 'получение образования'
  
display(credit_purpose ('покупка жилья')) #проверим работу функции
display(credit_purpose('получение высшего образования'))
credit_purpose('сыграть свадьбу')

'операции с недвижимостью'

'получение образования'

'проведение свадьбы'

In [27]:
bank_clients['purpose_category'] = bank_clients['purpose'].apply(credit_purpose) #применим функцию к датафрейму
bank_clients = bank_clients.drop(columns=['purpose']).reset_index(drop=True) #удалим из таблицы лишнюю колонку
bank_clients.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,total_income_category,purpose_category
0,1,8437.673028,42,0,0,f,сотрудник,0,253875,B,операции с недвижимостью
1,1,4024.803754,36,1,0,f,сотрудник,0,112080,C,операции с автомобилем
2,0,5623.422610,33,1,0,m,сотрудник,0,145885,C,операции с недвижимостью
3,3,4124.747207,32,1,0,m,сотрудник,0,267628,B,получение образования
4,0,340266.072047,53,1,1,f,пенсионер,0,158616,C,проведение свадьбы
5,0,926.185831,27,0,1,m,компаньон,0,255763,B,операции с недвижимостью
6,0,2879.202052,43,0,0,f,компаньон,0,240525,B,операции с недвижимостью
7,0,152.779569,50,1,0,m,сотрудник,0,135823,C,получение образования
8,2,6929.865299,35,0,1,f,сотрудник,0,95856,C,проведение свадьбы
9,0,2188.756445,41,1,0,m,сотрудник,0,144425,C,операции с недвижимостью


[Содержание](#0)

## Ответы на вопросы
<a id=3> </a>

### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [28]:
bank_clients_children = bank_clients.pivot_table(index='children', values='debt', aggfunc='mean').sort_values(by='debt')
#построим сводную таблицу со средним долгом относительно количества детей. Сделаем сортировку по величине долга
bank_clients_children

,debt
children,
5,0.000000
0,0.075444
3,0.081818
1,0.091658
2,0.094925
4,0.097561


Как видим - меньше всего долгов у бездетных. У клиентов с 5 детьми совсем нет долгов. Но прямой зависимости между количеством детей и долгом - нет, у клиентов с 1 ребенком в среднем чаще бывают долги, чем у клиентов с 3 детьми.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [29]:
bank_clients_family = bank_clients.pivot_table(index='family_status_id', values='debt', aggfunc='mean').sort_values(by='debt')
#построим сводную таблицу зависимости долга от семейного статуса
bank_family_status['family_status']=['женат / замужем', 'гражданский брак', 'вдовец / вдова', 'в разводе', 'не женат / не замужем']
#добавим в таблицу, в которой мы определяли семейные статусы колонку
bank_clients_family = bank_clients_family.merge(bank_family_status, on='family_status_id', how='left')
#объеденим 2 таблицы
bank_clients_family


,family_status_id,debt,family_status
0,2,0.065693,вдовец / вдова
1,3,0.071130,в разводе
2,0,0.075452,женат / замужем
3,1,0.093494,гражданский брак
4,4,0.097509,не женат / не замужем


Получается, у семейных людей, в среднем, меньше долгов по кредитам

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [30]:
bank_clients.pivot_table(index='total_income_category', values='debt', aggfunc='mean').sort_values(by='debt')
#построим сводную таблицу зависимости долга от среднего дохода

,debt
total_income_category,
D,0.060000
B,0.070649
A,0.076923
C,0.084915
E,0.090909


Получается, люди с самым низким доходом - до 30 000 р в месяц - чаще всего не возвращают кредит в срок.
Но прямой зависимости между уровнем дохода и возвратом кредита так же нет - люди с самыми высокими доходами не обязательно вернут кредит.

### Как разные цели кредита влияют на его возврат в срок?


In [31]:
bank_clients.pivot_table(index='purpose_category', values='debt', aggfunc='mean').sort_values(by='debt', ascending=False)
#составим сводную таблицу распределения долгов по разным категориям, отсортируем данные по убыванию 

,debt
purpose_category,
операции с автомобилем,0.093590
получение образования,0.092200
проведение свадьбы,0.080034
операции с недвижимостью,0.072340


Можно сделать вывод, что чаще всего не возвращают кредит те, кто берет его на нужды, связанные с автомобилем, а возвращают чаще всего - при операциях с недвижимостью. 

[Содержание](#0)

##  Вывод
<a id=4> </a>

Мы проанализировали предоставленные данные о клиентах банка и пришли к следующим выводам:

1. Клиенты без детей чаще всего возвращают кредит в срок.
2. Клиенты с наименьшим доходом, до 30000 р в месяц, чаще всего не вернут кредит.
3. Больше всего долгов по кредитам на нужды, связанные с автомобилем.
4. Меньше всего долгов по кредитам, связанным с недвижимостью.
5. У семейных людей и у тех, кто состоял в браке - меньше всего долгов по кредитам.
6. Количество детей не влияет напрямую на возврат кредита в срок.
7. Размер дохода не влияет напрямую на то, отдаст ли клиент кредит в срок.